In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 21.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845500 sha256=ce3280910f0a5a76a0ab9d2c8dc90e600de470a4bad764fef77cb0034715836d
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
!cd /usr/local/lib/python3.10/dist-packages/pyspark/jars && sudo wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar

--2023-07-06 05:02:46--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.60.104, 52.216.177.141, 52.217.236.216, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.60.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1048241 (1024K) [binary/octet-stream]
Saving to: ‘redshift-jdbc42-2.1.0.14.jar’

redshift-jdbc42-2.1 100%[===================>]   1024K  --.-KB/s    in 0.01s   

2023-07-06 05:02:46 (82.6 MB/s) - ‘redshift-jdbc42-2.1.0.14.jar’ saved [1048241/1048241]



In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL #1") \
    .getOrCreate()

In [4]:
rows_test = [
    { 'value': 1, 'name': 'Luka' },
    { 'value': 2, 'name': 'Luka' },
    { 'value': 3, 'name': 'Dirk' },
    { 'value': 4, 'name': 'Dirk' },
    { 'value': 5, 'name': 'Luka' },
]

df = spark.createDataFrame(rows_test)

In [5]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- value: long (nullable = true)



In [6]:
df.show()

+----+-----+
|name|value|
+----+-----+
|Luka|    1|
|Luka|    2|
|Dirk|    3|
|Dirk|    4|
|Luka|    5|
+----+-----+



In [7]:
df.createOrReplaceTempView("rows_test")

In [8]:
spark.sql("""
  SELECT
    value,
    SUM(value) OVER (
        order by value
        rows between 2 preceding and 2 following
    ) AS rolling_sum
  FROM rows_test""").show()

+-----+-----------+
|value|rolling_sum|
+-----+-----------+
|    1|          6|
|    2|         10|
|    3|         15|
|    4|         14|
|    5|         12|
+-----+-----------+



In [9]:
spark.sql("""
  SELECT
    value,
    SUM(value) OVER (
        order by value
        rows between unbounded preceding and 2 following
    ) AS rolling_sum
  FROM rows_test""").show()

+-----+-----------+
|value|rolling_sum|
+-----+-----------+
|    1|          6|
|    2|         10|
|    3|         15|
|    4|         15|
|    5|         15|
+-----+-----------+



In [10]:
spark.sql("""
  SELECT
    *,
    FIRST_VALUE(value) OVER (
        partition by name
        order by value
        rows between unbounded preceding and unbounded following
    ) AS min_value,
    LAST_VALUE(value) OVER (
        partition by name
        order by value
        rows between unbounded preceding and unbounded following
    ) AS max_value
  FROM rows_test""").show()

+----+-----+---------+---------+
|name|value|min_value|max_value|
+----+-----+---------+---------+
|Dirk|    3|        3|        4|
|Dirk|    4|        3|        4|
|Luka|    1|        1|        5|
|Luka|    2|        1|        5|
|Luka|    5|        1|        5|
+----+-----+---------+---------+



## Redshift 상의 다음 테이블을 데이터프레임으로 로딩하기
user_session_channel, session_timestamp, session_transaction

In [11]:
# Redshift와 연결해서 DataFrame으로 로딩하기
url = "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234"

df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

df_session_transaction = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_transaction") \
    .load()

In [12]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")
df_session_timestamp.createOrReplaceTempView("session_timestamp")
df_session_transaction.createOrReplaceTempView("session_transaction")

In [13]:
df_user_session_channel.show(5)

+------+--------------------+---------+
|userid|           sessionid|  channel|
+------+--------------------+---------+
|  1651|0004289ee1c7b8b08...|  Organic|
|  1197|00053f5e11d1fe4e4...| Facebook|
|  1401|00056c20eb5a02958...| Facebook|
|  1399|00063cb5da1826feb...| Facebook|
|  1667|000958fdaefe0dd06...|Instagram|
+------+--------------------+---------+
only showing top 5 rows



In [14]:
df_session_timestamp.show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|0002ac0d783338cfe...|2019-07-29 12:39:...|
|00053f5e11d1fe4e4...|2019-06-24 13:04:...|
|00056c20eb5a02958...| 2019-09-26 14:50:17|
|00063cb5da1826feb...|2019-10-16 14:04:...|
|0007cda84fafdcf42...|2019-05-22 08:02:...|
+--------------------+--------------------+
only showing top 5 rows



In [15]:
df_session_transaction.show(5)

+--------------------+--------+------+
|           sessionid|refunded|amount|
+--------------------+--------+------+
|00029153d12ae1c9a...|   false|    85|
|008909bd27b680698...|   false|    13|
|0107acb41ef20db22...|   false|    16|
|018544a2c48077d2c...|   false|    39|
|020c38173caff0203...|   false|    61|
+--------------------+--------+------+
only showing top 5 rows



## 위의 테이블들을 이용해서 사용자별로 처음 채널과 마지막 채널 알아내기

In [16]:
first_last_channel_df = spark.sql("""
WITH RECORD AS (
  SELECT /*사용자의 유입에 따른, 채널 순서 매기는 쿼리*/
      userid,
      channel,
      ROW_NUMBER() OVER (PARTITION BY userid ORDER BY ts ASC) AS seq_first,
      ROW_NUMBER() OVER (PARTITION BY userid ORDER BY ts DESC) AS seq_last
  FROM user_session_channel u
  LEFT JOIN session_timestamp t
    ON u.sessionid = t.sessionid
)
SELECT /*유저의 첫번째 유입채널, 마지막 유입 채널 구하기*/
      f.userid,
      f.channel first_channel,
      l.channel last_channel
FROM RECORD f
INNER JOIN RECORD l ON f.userid = l.userid
WHERE f.seq_first = 1 and l.seq_last = 1
ORDER BY userid
""")

In [ ]:
first_last_channel_df.show(10)

+------+-------------+------------+
|userid|first_channel|last_channel|
+------+-------------+------------+
|    27|      Youtube|   Instagram|
|    29|        Naver|       Naver|
|    33|       Google|     Youtube|
|    34|      Youtube|       Naver|
|    36|        Naver|     Youtube|
|    40|      Youtube|      Google|
|    41|     Facebook|     Youtube|
|    44|        Naver|   Instagram|
|    45|      Youtube|   Instagram|
|    59|    Instagram|   Instagram|
+------+-------------+------------+
only showing top 10 rows



In [ ]:
first_last_channel_df2 = spark.sql("""
SELECT DISTINCT A.userid,
    FIRST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS First_Channel,
    LAST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS Last_Channel
FROM user_session_channel A
LEFT JOIN session_timestamp B
ON A.sessionid = B.sessionid""")

In [ ]:
first_last_channel_df2.show()

+------+-------------+------------+
|userid|First_Channel|Last_Channel|
+------+-------------+------------+
|    27|      Youtube|   Instagram|
|    29|        Naver|       Naver|
|    33|       Google|     Youtube|
|    34|      Youtube|       Naver|
|    36|        Naver|     Youtube|
|    40|      Youtube|      Google|
|    41|     Facebook|     Youtube|
|    44|        Naver|   Instagram|
|    45|      Youtube|   Instagram|
|    59|    Instagram|   Instagram|
|    64|      Youtube|     Youtube|
|    65|      Youtube|     Organic|
|    68|      Youtube|     Organic|
|    69|     Facebook|   Instagram|
|    80|      Organic|       Naver|
|    84|       Google|     Youtube|
|    87|      Youtube|      Google|
|    97|      Organic|     Organic|
|   112|     Facebook|     Youtube|
|   113|      Organic|     Organic|
+------+-------------+------------+
only showing top 20 rows

